# 02 - Preprocesamiento y Feature Engineering

**Objetivo:** En este notebook, se prepara el dataset para el entrenamiento de modelos. Se aplican las decisiones tomadas en el EDA, se seleccionan las características más importantes, se dividen los datos y se escalan para dejarlos en un formato óptimo para los algoritmos de Machine Learning.

## 1. Carga de Datos e Importaciones
Importamos las librerías necesarias y cargamos el dataset limpio y optimizado que generamos en el primer notebook.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from joblib import dump
import warnings

In [2]:
# Ignoramos este tipo específico de RuntimeWarning que proviene del módulo de formato de Pandas,
# ya que sabemos que es un efecto secundario inofensivo de nuestra optimización de memoria.
warnings.filterwarnings(
    "ignore",
    category = RuntimeWarning,
    module = "pandas.io.formats.format"
)

# Cargar el dataset optimizado
df = pd.read_parquet(r'../data/processed/cic_ids_2017_optimized.parquet')

# Mostrar las primeras 5 filas para verificar que la carga fue exitosa
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


## 2. Selección Manual de Características
Basado en la alta correlación (0.89) encontrada en el EDA entre `Flow Duration` y `Flow IAT Mean`, eliminamos esta última para reducir la redundancia en el modelo.

In [3]:
# Eliminar la columna redundante identificada en el EDA
df.drop(columns = ['Flow IAT Mean'], inplace = True)

# Confirmar la nueva forma del DataFrame
print("Columna 'Flow IAT Mean' eliminada.")
print(f"Nuevas dimensiones del DataFrame: {df.shape}")

Columna 'Flow IAT Mean' eliminada.
Nuevas dimensiones del DataFrame: (2827876, 78)


## 3. División en Características (X) y Objetivo (y)
Separamos nuestro DataFrame en dos: 
- `X`: Contendrá todas las características predictoras.
- `y`: Contendrá la columna objetivo que queremos predecir (`Label`).

In [4]:
X = df.drop('Label', axis = 1)
y = df['Label']

# Imprimir las dimensiones para verificar la separación
print("Dimensiones de X (características):", X.shape)
print("Dimensiones de y (objetivo):", y.shape)

Dimensiones de X (características): (2827876, 77)
Dimensiones de y (objetivo): (2827876,)


## 4. División en Conjuntos de Entrenamiento y Prueba (Train/Test Split)
Dividimos los datos en un conjunto de entrenamiento (70%) y uno de prueba (30%). Es crucial hacer esto **antes** de aplicar técnicas como la selección automática o el escalado para evitar la fuga de datos (data leakage).

Se utiliza `stratify = y` para asegurar que la proporción de clases (el desbalance) sea la misma tanto en el conjunto de entrenamiento como en el de prueba.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

# Imprimir las dimensiones de los nuevos conjuntos de datos para verificar
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de y_test:", y_test.shape)

Dimensiones de X_train: (1979513, 77)
Dimensiones de X_test: (848363, 77)
Dimensiones de y_train: (1979513,)
Dimensiones de y_test: (848363,)


## 5. Selección Automática de Características

Utilizamos un modelo `RandomForestClassifier` como "juez" a través de `SelectFromModel` para evaluar la importancia de las características. Este método nos permite quedarnos solo con las variables más informativas, reduciendo la dimensionalidad del problema.

Se establece un umbral (`threshold = 'median'`) para seleccionar automáticamente la mitad superior de las características más importantes.

In [6]:
# Crear y entrenar el selector de características
selector_model = RandomForestClassifier(n_estimators = 50, random_state = 42, n_jobs = -1)
selector = SelectFromModel(estimator = selector_model, threshold = 'median', prefit = False) # prefit=False es buena práctica

print("Entrenando el selector de características (puede tardar unos minutos)...")
# El selector se ajusta solo con los datos de entrenamiento
selector.fit(X_train, y_train)

# Obtener los nombres de las columnas que nos vamos a quedar
selected_features = X_train.columns[selector.get_support()]

# Sobrescribir X_train y X_test para que contengan solo las características seleccionadas
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)

# Imprimir los resultados de la selección
print("\nProceso de selección completado.")
print("Número original de características:", len(selector.feature_names_in_))
print("Número de características seleccionadas:", X_train.shape[1])
print("\nCaracterísticas seleccionadas:")
print(selected_features.tolist())

Entrenando el selector de características (puede tardar unos minutos)...

Proceso de selección completado.
Número original de características: 77
Número de características seleccionadas: 39

Características seleccionadas:
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Max', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'PSH Flag Count', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Header Length.1', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward

## 6. Escalado de Características Numéricas (Scaling)

Aplicamos un `StandardScaler` para estandarizar las características (media 0, desviación estándar 1). Esto es crucial para el rendimiento de muchos algoritmos de Machine Learning.

El escalador se "ajusta" y "transforma" (`.fit_transform()`) con los datos de entrenamiento. Luego, se usa ese mismo escalador ya "entrenado" para aplicar la misma transformación (`.transform()`) al conjunto de prueba, evitando así la fuga de datos.

In [7]:
# Instanciar el escalador
scaler = StandardScaler()

# Ajustar y transformar el conjunto de entrenamiento en un solo paso
X_train_scaled = scaler.fit_transform(X_train)

# Transformar el conjunto de prueba usando los parámetros aprendidos del conjunto de entrenamiento
X_test_scaled = scaler.transform(X_test)

# Verificar las dimensiones (deberían ser las mismas que antes del escalado)
print("Dimensiones de X_train después del escalado:", X_train_scaled.shape)
print("Dimensiones de X_test después del escalado:", X_test_scaled.shape)

Dimensiones de X_train después del escalado: (1979513, 39)
Dimensiones de X_test después del escalado: (848363, 39)


## 7. Guardado de los Datos Preprocesados

Finalmente, guardamos todos los objetos que hemos creado (los conjuntos de entrenamiento y prueba ya escalados, las etiquetas y la lista de características seleccionadas) en un único archivo `.joblib`.

Esto nos permitirá cargar todos estos datos listos para usar con una sola línea de código en el notebook de modelado.

In [8]:
# Crear un diccionario con todos los objetos a guardar
datos_preprocesados = {
    'X_train_scaled': X_train_scaled,
    'X_test_scaled': X_test_scaled,
    'y_train': y_train,
    'y_test': y_test,
    'selected_features': selected_features # Usamos el nombre de variable consistente
}

# Definir la ruta de guardado
path_to_save = r'../data/processed/cicids2017_preprocessed_data.joblib'

# Guardar el diccionario en un archivo
dump(datos_preprocesados, path_to_save)

print(f"Datos preprocesados y guardados exitosamente en: {path_to_save}")

Datos preprocesados y guardados exitosamente en: ../data/processed/cicids2017_preprocessed_data.joblib
